In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import DistilBertForSequenceClassification, DistilBertTokenizer

from datasets.dataset_ver8 import DatasetGeneratorVer8
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main_ver8 import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

In [3]:
postfix = "distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_2"
outdir = Path("results/distilbert-base-uncased-ver8")
fold = 2

In [4]:
with open("config/distilbert-base-uncased-ver8.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [5]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased-ver8'),
 'debug': False,
 'seed': 20210425,
 'ver': 8,
 'train': {'SAM': False,
  'folds': [5],
  'checkpoints': [None],
  'loss': {'name': 'focal',
   'params': {'gamma': 2.0, 's': 45.0, 'm': 0.1, 'crit': 'focal'}},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 12,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ver6'),
  'batch_size': 35,
  'num_workers': 8,
  'oversampling': True,
  'oversampling_scale': 10,
  'augmentation': {'EDA_RD': True}}}

In [6]:
trainer = MyTrainer(C, fold, outdir / f"{postfix}.pth")

Load pretrained results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_2.pth


In [7]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [8]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [9]:
model.module.pre_classifier.register_forward_hook(hook)

In [10]:
train_dl = trainer.dsgen.train_only()

# 덱만들기 (Train)

In [11]:
import time

time.sleep(3600 * 3)

In [12]:
activation = []
deck = {"feat": [], "otext": [], "tlevel": [], "fclevel": []}
with tqdm(total=len(train_dl.dataset), ncols=100, file=sys.stdout) as t:
    for text, tlevel, otext in train_dl:
        fclevel = model(text.cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["tlevel"].append(tlevel)
        deck["otext"].extend(otext)
        t.update(text.size(0))

100%|██████████████████████████████████████████████████████| 421079/421079 [20:59<00:00, 334.29it/s]


In [13]:
deck["feat"] = torch.cat(activation)
deck["tlevel"] = torch.cat(deck["tlevel"])
deck["fclevel"] = torch.cat(deck["fclevel"])

In [14]:
deck["tlevel"].shape, deck["fclevel"].shape

(torch.Size([421079]), torch.Size([421079]))

In [15]:
deck["feat"].shape

torch.Size([421079, 768])

In [16]:
torch.save(deck, outdir / f"{postfix}-deck1.pth")

# Valid Lv7에 대한 결과 분석

In [17]:
deck = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_2-deck1.pth")

In [18]:
ds_lv7 = trainer.dsgen.valid_lv7().dataset

In [19]:
text, otext = ds_lv7[0]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-3.1733,  5.4342, -6.6915, -4.8008, -9.0336, -4.5570, -8.7731]])


In [20]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[41.0873, 47.1770, 50.5478, 53.1364]]),
 tensor([[375, 318, 753,  43]]),
 tensor([[0, 0, 1, 0]]))

In [21]:
text, otext = ds_lv7[1]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-4.1568, -2.9555, -9.2036,  2.0651, -8.0837, -2.1970, -7.7339]])


In [22]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[47.1414, 66.1885, 67.6206, 73.5886]]),
 tensor([[318, 589, 267, 495]]),
 tensor([[0, 0, 0, 0]]))

In [23]:
text, otext = ds_lv7[2]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[  2.4664,   0.6378, -10.0457,  -3.4297, -11.6076,  -4.1801, -10.3557]])


In [24]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[56.9343, 62.8391, 65.5041, 65.7527]]),
 tensor([[462, 605, 271, 410]]),
 tensor([[0, 0, 0, 0]]))

# 덱만들기 (Submission)

In [25]:
ds_test = trainer.dsgen.test()

In [26]:
activation = []
deck = {"feat": [], "otext": [], "fclevel": [], "ids": []}
with tqdm(total=len(ds_test), ncols=100, file=sys.stdout) as t:
    for i in range(len(ds_test)):
        text, otext, ids = ds_test[i]
        fclevel = model(text[None].cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["otext"].append(otext)
        deck["ids"].append(ids)
        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [1:50:28<00:00, 165.34it/s]


In [27]:
deck["feat"] = torch.stack(activation)
deck["fclevel"] = torch.stack(deck["fclevel"])

In [28]:
deck["feat"] = deck["feat"][:, 0, :]
deck["fclevel"] = deck["fclevel"][:, 0]

In [29]:
deck["feat"].shape, deck["fclevel"].shape

(torch.Size([1095951, 768]), torch.Size([1095951]))

In [30]:
torch.save(deck, outdir / f"{postfix}-deck2.pth")